### Aufgabe 18

Bisher haben wir uns lediglich lineare Klassifikatoren angeschaut. Jetzt möchten wir uns auch nicht-lineare Klassifikatoren anschauen. Aber dann wird es schwierig, immer die Ableitung zu berechnen. Da hilft uns Pytorch. Pytorch ist ganz ähnlich zu Numpy, ist allerdings für Maschinelles Lernen optimiert und kann zum Beispiel Gradienten selbstständig berechnen. Unten seht ihr ein komplettes Beispiel, wie man mit Hilfe von Pytorch ein lineares Modell wie im vorherigen Schritt optimieren kann. Versucht die besten Lernraten lr und die richtige Anzahl an Schritten zu finden, damit ihr euer Klassifikator eine Genauigkeit von über 80% bekommt.

In [75]:
import numpy as np

# Daten laden
D = np.load('data/train_data.npy')

In [84]:
import torch

def classifier(D, D_test):
    # konvertiere die Daten in einen Torch Tensor (getrennt in Eigenschaften und Labels)
    X = np.hstack([np.ones_like(D[:, 0])[:, None], D[:, :-1]])
    X = torch.from_numpy(X).float()
    y = torch.from_numpy(D[:, -1]).float()

    # definiere die Parameter
    w = torch.randn(14, requires_grad=True)

    # definiere die Kostenfunktion
    criterion = torch.nn.BCEWithLogitsLoss()

    # definiere den Gradientabstieg
    optimizer = torch.optim.SGD([w], lr=0.0001, momentum=0.9)

    for i in range(10000):   # loop over the dataset multiple times

        # Stelle sicher, dass keine Gradienten aus vorherigen Schritten gecached sind.
        optimizer.zero_grad()

        # Berechne den Output von deinem Modell
        # Diesmal ohne Sigmoid, das ist bei Torch in der Kostenfunktion bereits enthalten
        outputs = torch.matmul(X, w)

        # Wert der Kostenfunktion
        kosten = criterion(outputs, y)

        # Berechne die Gradienten für die Parameter
        kosten.backward()

        # Ändere die Parameter in Richtung der Gradienten
        optimizer.step()

        # Vorhergesagte labels
        yhat = outputs.detach().numpy() > 0

        # Messe die Genauigkeit des Klassifikators
        acc = np.mean(yhat == y.numpy())

        print(f'Iteration {i}: Kosten {kosten.item()}, Genauigkeit {acc}')
        
    # am Ende versuchen wir die Datenpunkte in D_test zu klassifizieren und auszugeben
    X_test = np.hstack([np.ones_like(D_test[:, 0])[:, None], D_test])
    X_test = torch.from_numpy(X_test).float()
    outputs_test = torch.matmul(X_test, w)
    
    labels_test = outputs_test.numpy() > 0
    
    return labels_test